# Analysis pipeline for Specificity

## Preliminaries

In [41]:
%cd ~/code/textrec/

/Users/kcarnold/code/textrec


In [42]:
import datetime
import json
import pathlib

import pandas as pd
import numpy as np
import toolz
from IPython.display import Image, HTML

In [3]:
from textrec.paths import paths
from textrec import analysis_util, util
reload(analysis_util), reload(util)

(<module 'textrec.analysis_util' from '/Users/kcarnold/code/textrec/src/textrec/analysis_util.py'>,
 <module 'textrec.util' from '/Users/kcarnold/code/textrec/src/textrec/util.py'>)

Download Karpathy's version of the COCO captions dataset. This has the train-test split that is more commonly used in the literature, as well as pre-tokenized captions.

In [4]:
images = util.get_coco_captions()
images_by_split = toolz.groupby('split', images)

In [5]:
id2img = {img['cocoid']: img for img in images}

In [6]:
id2url = util.get_coco_id2url()

In [7]:
def show_images(indices):
    def img(idx):
        img = id2img[idx]
        captions = '\n'.join(
            '<div>{}</div>'.format(sent)
            for sent in toolz.pluck('raw', img['sentences'])
        )
        return '<div style="display: inline-block;"><div>{}/{}</div><img src="{}">{}</div>'.format(
            img['split'], img['cocoid'], id2url[img['cocoid']], captions)

    return '\n'.join(img(idx) for idx in indices)

In [8]:
HTML(show_images([images_by_split['val'][0]['cocoid']]))

## Load results of writing experiment

In [9]:
%run -m textrec.batch_analysis

In [10]:
participants = get_participants_by_batch()

In [11]:
participants.keys()

dict_keys(['2018-04-09', '2018-04-24', '2018-04-27', '2018-05-02'])

In [12]:
# summarize('2018-04-27')
summarize('2018-05-02')


h52x67
practice-0:specific:a black cat napping on a sunny unpainted wood bench in front of a red wall
final-0-0:specific:a cat sitting next to a glass bowl, looking up to the camera
final-0-1:specific:a shower with dirty glass doors has a beige towel hanging on the outside
final-0-2:specific:there is no image here 
practice-1:norecs:a man with black hair and glasses placing a large turkey into an upper oven
final-1-0:norecs:a bride and groom cutting their wedding cake, while a photographer guides them
final-1-1:norecs:a man helping his children fly a multicolor butterfly kite on a clear day
final-1-2:norecs:a passenger train approaching a small quaint station with a blue and white building on the background
practice-2:general:a black and red vehicle with bikes on top and people standing nearby with umbrellas. 
final-2-0:general:a busy street in a historic town with a red bus driving on the street. 
final-2-1:general:a tennis player hits a ball during a game 
final-2-2:general:a surfer

_____________________________________________get_log_analysis_raw - 3.8s, 0.1min
practice-0:norecs:a black cat napping on a sunny unpainted wood bench in front of a red wall
final-0-0:norecs:a glass of red wine sitting in fronf of a brown cat with brown stripes on a brown mat
final-0-1:norecs:a beige towel hangs over the rightmost shower door both of which are wet with water
final-0-2:norecs:a toilet with the seat down and a roll of toilet paper on top is next to a white sink
practice-1:specific:a man with black hair and glasses placing a large turkey into an upper oven
final-1-0:specific:a husband, bride and female all stand in front of a table holding a knife cutting a cake
final-1-1:specific:two children and one adult stand on the beach holding kites and flying them into the sky 
final-1-2:specific:a train on the tracks passes a white house with blue paint before approaching another white house
practice-2:general:a black and red vehicle with bikes on top and people standing nearby w

In [13]:
trial_data = get_trial_data('2018-05-02')

In [14]:
for trial in trial_data:
    trial['text'] = trial['text'].strip()

I had the wrong URL for one image when one person ran it.

In [15]:
trial_data = [trial for trial in trial_data if not (trial['stimulus'] == 431140 and trial['participant'] == 'h52x67')]

In [16]:
list(toolz.pluck('text', trial_data))

['a cat sitting next to a glass bowl, looking up to the camera',
 'a shower with dirty glass doors has a beige towel hanging on the outside',
 'a bride and groom cutting their wedding cake, while a photographer guides them',
 'a man helping his children fly a multicolor butterfly kite on a clear day',
 'a passenger train approaching a small quaint station with a blue and white building on the background',
 'a busy street in a historic town with a red bus driving on the street.',
 'a tennis player hits a ball during a game',
 'a surfer riding a wave on a sunny day',
 'a yellow cat with strips is setting on a place mat  on a table behind a half of glass of red wine',
 'a small bath with a shower with a blue mat on the floor',
 'a bath room with a white toilet and a white washbasin',
 'a newly wedded couple cutting there wedding cake with a lady helping the bride cut the cake',
 'a group of people flying kites on a beach and its a beautiful day',
 'a train station with three building to t

In [42]:
# pd.DataFrame(trial_data).to_clipboard()

# Generate nAFC task

In [17]:
cap_vectorizer, caption_vecs = util.get_vectorized_captions()

In [18]:
caption_vecs.shape

(123287, 9952)

In [19]:
id2url[570528]

'http://images.cocodataset.org/train2017/000000570528.jpg'

In [20]:
def get_similar_images(caption, n=10):
    query_vec = cap_vectorizer.transform([caption])
    similarity = caption_vecs.dot(query_vec.T).A.ravel()
    return [images[idx]['cocoid'] for idx in np.argsort(similarity)[-n:][::-1]]
query_caption = trial_data[0]['text']
print(query_caption)
HTML(show_images(get_similar_images(query_caption)))

a cat sitting next to a glass bowl, looking up to the camera


In [21]:
'|'.join(id2url[idx] for idx in get_similar_images(query_caption))

'http://images.cocodataset.org/train2017/000000570528.jpg|http://images.cocodataset.org/train2017/000000300732.jpg|http://images.cocodataset.org/train2017/000000400999.jpg|http://images.cocodataset.org/train2017/000000449770.jpg|http://images.cocodataset.org/train2017/000000146767.jpg|http://images.cocodataset.org/train2017/000000548661.jpg|http://images.cocodataset.org/train2017/000000279386.jpg|http://images.cocodataset.org/train2017/000000350727.jpg|http://images.cocodataset.org/train2017/000000002295.jpg|http://images.cocodataset.org/train2017/000000537417.jpg'

In [22]:
def get_foil_set(*, stimulus, caption, rs):
    similar_images = get_similar_images(caption, n=10)
    if stimulus not in similar_images:
        print("Inserting", stimulus, 'into foil set')
        similar_images[-1] = stimulus
    rs.shuffle(similar_images)
    return similar_images
trial = trial_data[1]
get_foil_set(stimulus=trial['stimulus'], caption=trial['text'], rs=np.random.RandomState(1234))

[396295, 277466, 362711, 212082, 409842, 233737, 510852, 503200, 480798, 98257]

In [84]:
def make_task(trial, rs):
    stimulus = trial['stimulus']
    foil_set = get_foil_set(stimulus=stimulus, caption=trial['text'], rs=rs)
    res = dict(description=trial['text'], correct_idx=foil_set.index(stimulus))
    for i, idx in enumerate(foil_set):
        res[f'image_{i}_url'] = id2url[idx]
    return res

In [104]:
make_task(trial_data[0], np.random.RandomState(123))

Inserting 275449 into foil set


{'correct_idx': 8,
 'description': 'a cat sitting next to a glass bowl, looking up to the camera',
 'image_0_url': 'http://images.cocodataset.org/train2017/000000146767.jpg',
 'image_1_url': 'http://images.cocodataset.org/train2017/000000570528.jpg',
 'image_2_url': 'http://images.cocodataset.org/train2017/000000350727.jpg',
 'image_3_url': 'http://images.cocodataset.org/train2017/000000548661.jpg',
 'image_4_url': 'http://images.cocodataset.org/train2017/000000002295.jpg',
 'image_5_url': 'http://images.cocodataset.org/train2017/000000449770.jpg',
 'image_6_url': 'http://images.cocodataset.org/train2017/000000300732.jpg',
 'image_7_url': 'http://images.cocodataset.org/train2017/000000279386.jpg',
 'image_8_url': 'http://images.cocodataset.org/train2017/000000275449.jpg',
 'image_9_url': 'http://images.cocodataset.org/train2017/000000400999.jpg'}

### Load MTurk results so far

In [26]:
# result_files = list((paths.data / 'mturk').glob('*-2AFC.csv'))
# mturk_2afc_results = (
#     pd.concat([pd.read_csv(str(f)) for f in result_files], axis=0, ignore_index=True)
#     if len(result_files)
#     else pd.DataFrame([], columns=['Input.caption']))
result_files = list((paths.data / 'mturk').glob('*-nAFC.csv'))
mturk_nafc_results = (
    pd.concat([pd.read_csv(str(f)) for f in result_files], axis=0, ignore_index=True)
    if len(result_files)
    else pd.DataFrame([], columns=['Answer.description']))

In [27]:
num_comparisons_by_caption = mturk_nafc_results.groupby('Answer.description').size().to_dict()

### Generate tasks remaining

In [28]:
trials_todo = [trial for trial in trial_data if num_comparisons_by_caption.get(trial['text'], 0) < 5]
len(trial_data), len(trials_todo)

(71, 71)

In [29]:
i = 0
while True:
    out_fn = paths.top_level / 'HITs' / f'{datetime.date.today().isoformat()}-{i}-nAFC.csv'
    if not out_fn.exists():
        break
    i += 1
out_fn

PosixPath('/Users/kcarnold/code/textrec/HITs/2018-05-11-0-nAFC.csv')

In [130]:
rs = np.random.RandomState(1234)
pd.DataFrame([make_task(trial, rs) for trial in trials_todo[:5]]).to_csv(out_fn, index=False)

Inserting 275449 into foil set
Inserting 200451 into foil set
Inserting 223777 into foil set


### Generate the actual HIT text...

In [108]:
from jinja2 import Template
html = Template(open(paths.top_level / 'HITs' / '2018-05-04-image-description-match.jinja.html').read()).render(dict(
    description='${description}',
    images=['${image_%d_url}' % i for i in range(10)]))

In [113]:
html2 = html
for k, v in make_task(trial_data[2], np.random.RandomState(1234)).items():
    html2 = html2.replace('${' + k + '}', str(v))
HTML('<div style="height: 500px; position: relative;">'+html2+'</div>')
# print(html2)

In [129]:
import subprocess
subprocess.Popen('pbcopy', stdin=subprocess.PIPE).communicate(html.encode('utf-8'))

(None, None)

# Analyze MTurk results

In [49]:
mturk_2afc_results['picked_id'] = mturk_2afc_results.apply(
    lambda row: dict(optionA=row['Input.image_A_id'], optionB=row['Input.image_B_id'])[row['Answer.choice']], axis=1)
mturk_2afc_results['is_correct'] = mturk_2afc_results['picked_id'] == mturk_2afc_results['Input.stimulus_id']
mturk_2afc_results['is_correct']

0       True
1       True
2       True
3       True
4       True
5       True
6       True
7       True
8       True
9       True
10      True
11      True
12      True
13      True
14      True
15     False
16      True
17     False
18      True
19     False
20     False
21      True
22      True
23      True
24      True
25      True
26     False
27      True
28      True
29      True
       ...  
325     True
326     True
327     True
328     True
329     True
330     True
331     True
332     True
333     True
334     True
335     True
336     True
337     True
338     True
339     True
340     True
341     True
342     True
343     True
344     True
345     True
346     True
347     True
348     True
349     True
350     True
351     True
352     True
353    False
354    False
Name: is_correct, Length: 355, dtype: bool

In [50]:
specificity_lookup = mturk_2afc_results.groupby('Input.caption').is_correct.mean().to_dict()

In [51]:
specificity_lookup

{'a bath room with a white toilet and a white washbasin': 0.8,
 'a bathroom has a white toilet and matching sink.': 1.0,
 'a beige towel hangs over the rightmost shower door both of which are wet with water': 1.0,
 'a black and white photo or a large man and a woman cutting thwir wedding cake': 1.0,
 'a blonde man in a wetsuit surfs a wave.': 0.6,
 'a bride and groom are both holding onto the same knife to cut their wedding cake.': 0.6,
 'a bride and groom cutting their wedding cake, while a photographer guides them': 1.0,
 'a brown trai  pulls into the tracka next to some colorful buildings': 1.0,
 'a busy city street has sidewalks full of people and a bus and some cars.': 1.0,
 'a busy city street with cars and people along the streets with high-rise buildings on both sides': 1.0,
 'a busy city street with cars, a large red bus and pedestrians going about their day.': 1.0,
 'a busy street in a historic town with a red bus driving on the street.': 1.0,
 'a car is sitting on a table be

In [52]:
results = pd.DataFrame([dict(trial, specificity=specificity_lookup[trial['text'].strip()]) for trial in trial_data])
for col in ['condition', 'participant']:
    results[col] = results[col].astype('category')
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 7 columns):
block           71 non-null int64
condition       71 non-null category
idx_in_block    71 non-null int64
participant     71 non-null category
specificity     71 non-null float64
stimulus        71 non-null int64
text            71 non-null object
dtypes: category(2), float64(1), int64(3), object(1)
memory usage: 3.5+ KB


In [53]:
results.groupby('condition').specificity.mean()

condition
general     0.891667
norecs      0.808333
specific    0.808696
Name: specificity, dtype: float64

# How many images does this caption apply to?

In [35]:
data = pd.read_csv("/Users/kcarnold/Downloads/Submitted Captions - Sheet1.csv").iloc[:,:6]

In [45]:
data

,block,idx_in_block,condition,participant,text,number of images it applies to (1 min limit),is_unique
0,0,0,specific,2vvj5m,a plate with pancakes with bananas and bacon w...,1,True
1,0,1,specific,2vvj5m,a red bird and green bird about to eat out of ...,1,True
2,0,2,specific,2vvj5m,a man is watching a field of sheep eat grass.,some,False
3,1,0,general,2vvj5m,a man is standing with a very large colorful k...,some,False
4,1,1,general,2vvj5m,a very large kitchen with a nice big island wi...,some,False
5,1,2,general,2vvj5m,a black cat with big green eyes with something...,a couple,False
6,2,0,norecs,2vvj5m,a small white and blue bird perched on a branc...,"1 exact, a few close",False
7,2,1,norecs,2vvj5m,a young girl in a red top and black skirt play...,1,True
8,2,2,norecs,2vvj5m,a yellow light hanging above a pedestrian cros...,3,False
18,0,0,norecs,m2xcp6,breakfast that consists of pancakes banana bac...,1,True


In [36]:
data.shape

(54, 6)

In [37]:
data = data.dropna().copy()
data.shape

(45, 6)

In [38]:
data['is_unique'] = (data.iloc[:,5] == '1')
data.is_unique.mean()

0.37777777777777777

In [49]:
data.groupby(['block', 'idx_in_block', 'condition']).is_unique.mean()

block  idx_in_block  condition
0      0             general      1.0
                     norecs       1.0
                     specific     1.0
       1             general      1.0
                     norecs       1.0
                     specific     1.0
       2             general      0.0
                     norecs       0.0
                     specific     0.0
1      0             general      0.0
                     norecs       0.0
                     specific     0.5
       1             general      0.0
                     norecs       0.5
                     specific     0.0
       2             general      0.0
                     norecs       0.5
                     specific     0.5
2      0             general      0.0
                     norecs       0.0
                     specific     0.5
       1             general      0.0
                     norecs       0.5
                     specific     0.0
       2             general      1.0
                   

In [46]:
data['participant'] = data['participant'].astype('category')
data['condition'] = data['condition'].astype('category')

In [48]:
data.groupby('condition').is_unique.mean()

condition
general     0.466667
norecs      0.333333
specific    0.333333
Name: is_unique, dtype: float64

In [47]:
%%R -i data
transformed <- art(is_unique ~ condition + (1|participant), data=data)
summary(transformed)
anova(transformed)

Analysis of Variance of Aligned Rank Transformed Data

Table Type: Analysis of Deviance Table (Type III Wald F tests with Kenward-Roger df) 
Model: Mixed Effects (lmer)
Response: art(is_unique)

                  F Df Df.res Pr(>F)  
1 condition 0.36019  2     38 0.6999  
---
Signif. codes:   0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1 


In [26]:
example = trial_data[-1]['text']
example

'a yellow pedestrian crossing sign sittting below a signal light that is yellow. '

Concepts: traffic light. COCO doesn't have "pedestian crossing sign". There are 4330 images with traffic lights in them in COCO. That's way too much. Looking at Visual Genome.

Visual Genome synsets are potentially best, but they're sometimes inaccurate. e.g., "18 wheeler" is "cyclist.n.01". So let's consider an object a match if matches either the synset or object name.

In [30]:
vg_base = pathlib.Path('/Data/VisualGenome')
image_objects = json.load(open(vg_base / 'objects.json'))

In [31]:
img_by_id = {img['image_id']: img for img in image_objects}

In [37]:
obj_synsets = json.load(open(vg_base / 'object_synsets.json'))

In [63]:
obj_attributes = json.load(open(vg_base / 'attributes.json'))

In [66]:
len(obj_attributes), len(image_objects)

(108077, 108077)

In [68]:
obj_attributes[0].keys()

dict_keys(['image_id', 'attributes'])

In [70]:
attributes_by_img = {att['image_id']: att['attributes'] for att in obj_attributes}

In [71]:
attributes_by_img[61514]

[{'attributes': ['yellow', 'ellow'],
  'h': 247,
  'names': ['stop light', 'traffic signal'],
  'object_id': 1072926,
  'synsets': ['light.n.01'],
  'w': 102,
  'x': 393,
  'y': 1},
 {'h': 82,
  'names': ['yellow light'],
  'object_id': 1072927,
  'synsets': ['light.n.01'],
  'w': 89,
  'x': 393,
  'y': 81},
 {'attributes': ['yellow', 'warig', 'hagig', 'working', 'here'],
  'h': 76,
  'names': ['traffic light', 'light'],
  'object_id': 1072928,
  'synsets': ['traffic_light.n.01'],
  'w': 89,
  'x': 396,
  'y': 84},
 {'attributes': ['gloomy blue', 'blue', 'cloudy'],
  'h': 768,
  'names': ['sky'],
  'object_id': 1072929,
  'synsets': ['sky.n.01'],
  'w': 1020,
  'x': 1,
  'y': 0},
 {'attributes': ['here', 'horizontal'],
  'h': 432,
  'names': ['power lines', 'electric wires'],
  'object_id': 1072930,
  'synsets': ['power_line.n.01'],
  'w': 1022,
  'x': 1,
  'y': 0},
 {'attributes': ['black bolt',
   'yellow',
   'ellow',
   'diamod shaped',
   'here',
   'diamond shaped'],
  'h': 683,


In [74]:
# def has_object(obj_name):
#     return {
#         img['image_id'] for img in image_objects
#         if any(obj_name in '\n'.join(obj['names']) for obj in img['objects'])
#            }
def has_object(imgid, obj_name):
    return any(obj_name in '\n'.join(obj['names']) for obj in attributes_by_img[imgid])

In [75]:
# def has_synset(obj_synset):
#     return {
#         img['image_id'] for img in image_objects
#         if any(obj_synset in obj['synsets'] for obj in img['objects'])}
def has_synset(imgid, obj_synset):
    return any(obj_synset in obj['synsets'] for obj in attributes_by_img[imgid])

In [87]:
def has_obj_with_attr(imgid, obj_name, attr):
    return any(
        (obj_name in '\n'.join(obj['names'])) and (attr in obj.get('attributes', []))
        for obj in attributes_by_img[imgid])

def has_synset_with_attr(imgid, obj_synset, attr):
    return any(
        (obj_synset in obj['synsets']) and (attr in obj.get('attributes', []))
        for obj in attributes_by_img[imgid])

def has_obj_without_attr(imgid, obj_name, attr):
    return any(
        (obj_name in '\n'.join(obj['names'])) and (attr not in obj.get('attributes', []))
        for obj in attributes_by_img[imgid])

def has_synset_without_attr(imgid, obj_synset, attr):
    return any(
        (obj_synset in obj['synsets']) and (attr not in obj.get('attributes', []))
        for obj in attributes_by_img[imgid])

In [61]:
obj_synsets['pedestrian sign']

'sign.n.02'

In [88]:
# candidates = (
#     (has_object('pedestrian sign') | has_object('pedestrian crossing sign') | has_object('crossing sign') | has_object('sign')) &
#     (has_object('traffic light') | has_synset('traffic_light.n.01'))
# )
candidates = {
    imgid for imgid in attributes_by_img.keys()
    if (
        (
            has_object(imgid, 'pedestrian sign') |
            has_object(imgid, 'pedestrian crossing sign') |
            has_object(imgid, 'crossing sign') |
            has_obj_with_attr(imgid, 'sign', 'yellow')
        ) & (
            has_obj_without_attr(imgid, 'traffic light', 'red') |
            has_synset_without_attr(imgid, 'traffic_light.n.01', 'red')
        ))}
len(candidates)

246

In [91]:
show_images(candidates)

In [58]:
[img['image_id'] for img in image_objects if '61514' in img.get('image_url', '')]
#Image(img_by_id[61514]['image_url'])

[61514, 2361514]

In [55]:
img_by_id[61514]

{'image_id': 61514,
 'image_url': 'http://cs.stanford.edu/people/rak248/VG_100K_2/61514.jpg',
 'objects': [{'h': 21,
   'merged_object_ids': [],
   'names': ['bolt'],
   'object_id': 1072935,
   'synsets': ['thunderbolt.n.01'],
   'w': 42,
   'x': 595,
   'y': 123},
  {'h': 54,
   'merged_object_ids': [],
   'names': ['drawig of head'],
   'object_id': 1072939,
   'synsets': ['head.n.01'],
   'w': 37,
   'x': 577,
   'y': 152},
  {'h': 76,
   'merged_object_ids': [],
   'names': ['gree light'],
   'object_id': 1072937,
   'synsets': [],
   'w': 91,
   'x': 397,
   'y': 160},
  {'h': 206,
   'merged_object_ids': [],
   'names': ['lies'],
   'object_id': 1072936,
   'synsets': ['lie.n.01'],
   'w': 304,
   'x': 493,
   'y': 376},
  {'h': 56,
   'merged_object_ids': [],
   'names': ['pole'],
   'object_id': 1072940,
   'synsets': ['pole.n.01'],
   'w': 59,
   'x': 631,
   'y': 714},
  {'h': 432,
   'merged_object_ids': [],
   'names': ['power lines'],
   'object_id': 1072930,
   'synsets'

# Measuring Specificity

We use paired comparisons to analyze specificity and accuracy. For a target image $x$ and a fixed set of imposter images $Y$, the **specific accuracy** of a caption is the fraction of comparisons that chose $x$. 

We start with our dataset of paired comparisons.

In [17]:
captions = [
    "exactly how are both the dog and the person going to fit on that skateboard?",
    "the dark haired dog is trying to ride on the skateboard.",
    "a person in shorts and a black dog both have one foot on a skateboard.",
    "a dog with a black head and black legs and ears standing up has one black paw on a black skateboard with white wheels and a guy with black and white shoes and white socks has one foot on the skateboard also and there are bikes and other people in the background"
]

In [26]:
alternatives = 'dog-and-guy-on-skateboard just-dog-on-skateboard guy-on-skateboard-holding-dog dog-and-guy-next-to-skateboard'.split()
target = alternatives[0]
imposters = alternatives[1:]
applies_to = [
    'dog-and-guy-on-skateboard dog-and-guy-next-to-skateboard'.split(),
    'just-dog-on-skateboard'.split(),
    'dog-and-guy-on-skateboard'.split(),
    'dog-and-guy-on-skateboard just-dog-on-skateboard guy-on-skateboard-holding-dog dog-and-guy-next-to-skateboard'.split()
]
applies_to = {cap: tgts for cap, tgts in zip(captions, applies_to)}
applies_to

{'a dog with a black head and black legs and ears standing up has one black paw on a black skateboard with white wheels and a guy with black and white shoes and white socks has one foot on the skateboard also and there are bikes and other people in the background': ['dog-and-guy-on-skateboard',
  'just-dog-on-skateboard',
  'guy-on-skateboard-holding-dog',
  'dog-and-guy-next-to-skateboard'],
 'a person in shorts and a black dog both have one foot on a skateboard.': ['dog-and-guy-on-skateboard'],
 'exactly how are both the dog and the person going to fit on that skateboard?': ['dog-and-guy-on-skateboard',
  'dog-and-guy-next-to-skateboard'],
 'the dark haired dog is trying to ride on the skateboard.': ['just-dog-on-skateboard']}

In [40]:
import random
random.seed(0)
pairs = [[target, imposter] for imposter in imposters]
for pair in pairs:
    random.shuffle(pair)
pairs

[['dog-and-guy-on-skateboard', 'just-dog-on-skateboard'],
 ['dog-and-guy-on-skateboard', 'guy-on-skateboard-holding-dog'],
 ['dog-and-guy-next-to-skateboard', 'dog-and-guy-on-skateboard']]

In [41]:
def fake_answer_pairs_for_caption(applies, pairs):
    outcomes = []
    for a, b in pairs:
        choices = []
        if a in applies:
            choices.append(0)
        if b in applies:
            choices.append(1)
        if len(choices) == 0:
            choices = [0, 1]
        outcomes.append(random.choice(choices))
    return outcomes
fake_answer_pairs_for_caption(applies_to[captions[0]], pairs)

[0, 0, 1]

In [56]:
fake_comparisons_data = []
for caption in captions:
    for annotator in range(5):
        for pair, outcome in zip(pairs, fake_answer_pairs_for_caption(applies_to[caption], pairs)):
            picked = pair[outcome]
            fake_comparisons_data.append(dict(
                caption=caption,
                annotator=annotator,
                pair=pair,
                picked=picked))

In [57]:
data = pd.DataFrame(fake_comparisons_data)
len(data)

60

In [58]:
data['picked_correct'] = data['picked'] == 'dog-and-guy-on-skateboard'
data.groupby('caption').picked_correct.mean().sort_values()

caption
a dog with a black head and black legs and ears standing up has one black paw on a black skateboard with white wheels and a guy with black and white shoes and white socks has one foot on the skateboard also and there are bikes and other people in the background    0.266667
the dark haired dog is trying to ride on the skateboard.                                                                                                                                                                                                                 0.466667
exactly how are both the dog and the person going to fit on that skateboard?                                                                                                                                                                                             0.800000
a person in shorts and a black dog both have one foot on a skateboard.                                                                                                    

# Final analyses

We find a main effect of writing condition on outcome specificity.

In [37]:
%load_ext rpy2.ipython

In [12]:
results = pd.DataFrame([
    dict(participant_id=participant_id, condition=condition)
    for participant_id in 'abc def ghi'.split() for condition in 'general specific norecs'.split()
])
results['participant_id'] = results['participant_id'].astype('category')
results['condition'] = results['condition'].astype('category')
results['specificity'] = np.random.randn(len(results))

In [38]:
%%R
#install.packages("ARTool")
library(ARTool)

In [39]:
%%R -i results
summary(results)

     block          condition   idx_in_block     participant  specificity    
 Min.   :0.000   general :24   Min.   :0.0000   36x2r3 : 9   Min.   :0.2000  
 1st Qu.:0.000   norecs  :24   1st Qu.:0.0000   3vf5fg : 9   1st Qu.:0.7750  
 Median :1.000   specific:23   Median :1.0000   692c8j : 9   Median :1.0000  
 Mean   :1.014                 Mean   :0.9859   77j4mf : 9   Mean   :0.8359  
 3rd Qu.:2.000                 3rd Qu.:2.0000   gg65g6 : 9   3rd Qu.:1.0000  
 Max.   :2.000                 Max.   :2.0000   jvccx2 : 9   Max.   :1.0000  
                                                (Other):17                   
    stimulus          text          
 Min.   : 71815   Length:71         
 1st Qu.:223777   Class :character  
 Median :240275   Mode  :character  
 Mean   :254672                     
 3rd Qu.:275449                     
 Max.   :431140                     
                                    


In [41]:
%%R -i results
transformed <- art(specificity ~ condition + (1|participant), data=results)
summary(transformed)
anova(transformed)

Analysis of Variance of Aligned Rank Transformed Data

Table Type: Analysis of Deviance Table (Type III Wald F tests with Kenward-Roger df) 
Model: Mixed Effects (lmer)
Response: art(specificity)

                  F Df Df.res  Pr(>F)  
1 condition 0.90884  2 61.179 0.40836  
---
Signif. codes:   0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1 
